In [1]:
%load_ext autoreload
%autoreload 2

## Run the stack overflow data processing

In [ ]:
import data_cleanup_stack

# Note: the data can be downloaded from https://archive.org/download/stackexchange
input_file = 'cstheory.stackexchange.com/Posts.xml'
output_file = 'cstheory.jsonl'

In [ ]:
questions = data_cleanup_stack.read_xml(input_file)
questions = data_cleanup_stack.filter_questions(questions)
data_cleanup_stack.validate_data(questions)
data_cleanup_stack.write_jsonl(questions, output_file)

## Run the M1 data processing

In [2]:
# Import utils.py from parent directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils
import data_cleanup_m1

input_file = 'M1_preference_data_15052024.json'
output_file = 'dpo_m1.jsonl'

In [3]:
input_data = utils.read_json(input_file)
output_data = data_cleanup_m1.generate_preference_pairs(input_data)
utils.write_jsonl(output_data, output_file)